In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq
llm_model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000192D7EAA2C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000192D7EA8C40>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage, AIMessage
# llm_model.invoke([HumanMessage(content="Hi, my name is Yugal and I am incoming grad student at USC")])
llm_model.invoke(
    [
        HumanMessage(content="Hi, my name is Yugal and I am incoming grad student at USC"),
        AIMessage(content="Hi Yugal,\n\nWelcome to USC!  That's exciting!\n\nWhat program will you be joining?  \n\nI'm happy to help with any questions you might have about your time at USC, or just chat about your interests. \n\n"),
        HumanMessage(content="Hey, what is my name?")
    ]
)

AIMessage(content='Your name is Yugal.  \n\nI remember that you told me at the beginning of our conversation. 😊 \n\n\n\n', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 98, 'total_tokens': 125, 'completion_time': 0.054, 'prompt_time': 0.016826839, 'queue_time': None, 'total_time': 0.070826839}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-306ef614-23d6-4243-b8b7-401760ea3411-0', usage_metadata={'input_tokens': 98, 'output_tokens': 27, 'total_tokens': 125})

### Message history
We can use Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Further interactions will then load those messages and pass them into the chain as part of the input

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

with_msg_history = RunnableWithMessageHistory(llm_model, get_session_history)

In [10]:
config = {"configurable": {"session_id": "chat1"}}

In [12]:
response = with_msg_history.invoke(
    [HumanMessage(content="Hi my name is yugal, and I am a Software Engineer at Amazon/Microsoft/Google/Apple")],
    config=config
)

response.content

"Hi Yugal, it's great to meet you!  Being a Software Engineer at a company like Amazon, Microsoft, Google, or Apple is really something.  \n\nCould you tell me a little bit about what you work on?  I'm curious to hear about the kind of projects you're involved in.  \n\n"

In [13]:
with_msg_history.invoke(
    [HumanMessage(content="What is my name and where do I work?")],
    config=config
)

AIMessage(content="You said your name is Yugal and that you are a Software Engineer at one of these companies:\n\n* Amazon\n* Microsoft\n* Google\n* Apple \n\n\nLet me know if you'd like to tell me which one! 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 227, 'total_tokens': 281, 'completion_time': 0.108359015, 'prompt_time': 0.020072257, 'queue_time': None, 'total_time': 0.128431272}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d6eca81-a4cd-4829-935c-a3691674c60e-0', usage_metadata={'input_tokens': 227, 'output_tokens': 54, 'total_tokens': 281})

In [14]:
# change the config and see whether it tries to remember the chat history of chat1. (It should not as the session_id is changed and a new chat history will be created)
config = {"configurable": {"session_id": "chat2"}}
with_msg_history.invoke(
    [HumanMessage(content="What is my name and where do I work?")],
    config=config
)

AIMessage(content="As an AI, I have no memory of past conversations and no access to personal information about you, including your name or your workplace.\n\nIf you'd like to tell me your name and where you work, I'd be happy to know! 😊\n", response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 24, 'total_tokens': 79, 'completion_time': 0.110383174, 'prompt_time': 0.002544888, 'queue_time': None, 'total_time': 0.112928062}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-490d9c2f-cf69-4728-afc0-6df081999720-0', usage_metadata={'input_tokens': 24, 'output_tokens': 55, 'total_tokens': 79})

In [15]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi my name is yugal, and I am a Software Engineer at Amazon/Microsoft/Google/Apple'), AIMessage(content="Hi Yugal, it's nice to meet you!\n\nThat's impressive, working as a Software Engineer at one of those tech giants! Which one is it, if you don't mind me asking?  \n\nI'm always interested in hearing about people's experiences in the tech industry. What kind of work do you do? \n\n", response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 35, 'total_tokens': 111, 'completion_time': 0.153623564, 'prompt_time': 0.004633226, 'queue_time': None, 'total_time': 0.15825678999999998}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7eb37ded-9da5-43d7-940b-15842c3c320d-0', usage_metadata={'input_tokens': 35, 'output_tokens': 76, 'total_tokens': 111}), HumanMessage(content='Hi my name is yugal, and I am a Software Engineer at Amazon/

### Prompt Template

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message which we are passing to the LLM. 
First, lets add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides the messages.

In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|llm_model
chain.invoke({"messages": [HumanMessage(content="Hi, my name is Yugal")]})
with_msg_history = RunnableWithMessageHistory(chain, get_session_history)



In [36]:
config = {"configurable": {"session_id": "chat3"}}
response = with_msg_history.invoke(
    [HumanMessage(content="Hi my name is Yugal")],
    config=config
)

response

AIMessage(content="Hi Yugal, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 81, 'total_tokens': 109, 'completion_time': 0.056, 'prompt_time': 0.012488728, 'queue_time': None, 'total_time': 0.068488728}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-206f2f75-739d-434d-9fee-6dea79ca7452-0', usage_metadata={'input_tokens': 81, 'output_tokens': 28, 'total_tokens': 109})

In [37]:
complex_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages"),
])

complex_chain = complex_prompt|llm_model

In [38]:
response = complex_chain.invoke({"messages": [HumanMessage(content="Hi my name is Yugal")], "language": "Spanish"})
response.content

'¡Hola Yugal! Mucho gusto en conocerte. ¿Cómo puedo ayudarte hoy? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, becuase there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [39]:
with_msg_history = RunnableWithMessageHistory(complex_chain, get_session_history, input_messages_key="messages")

In [40]:
config = {"configurable": {"session_id": "chat4"}}
response = with_msg_history.invoke(
    {
        "messages": [HumanMessage(content="Hi i am Yugal")],
        "language": "Spanish"
    },
    config = config
)

response.content

'¡Hola Yugal! ¿Cómo estás? 😊  \n\n¿En qué te puedo ayudar hoy? \n'

In [42]:
response = with_msg_history.invoke(
    {
        "messages": [HumanMessage(content="What is my name?")],
        "language": "Spanish"
    },
    config = config
)
response.content

'Tu nombre es Yugal. 😊  \n\n¿Hay algo más que quieras saber? \n'

### Managing the Conversation history

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window.of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


`trim_messages` helps to reduce the how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with the other parameters like if we want to always keep the system message and whether to allow partial messages.

In [46]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(max_tokens=45, strategy="last", token_counter=llm_model, include_system=True, allow_partial=False, start_on="human")

messages = [
    SystemMessage(content="You are an expert assistant"),
    HumanMessage(content="Hi, I am John Doe"),
    AIMessage(content="Hey John Doe, how can I help you!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="2+2 = 4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem, what more can I help you with!"),
    HumanMessage(content="having fun"),
    AIMessage(content="Yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content='You are an expert assistant'),
 HumanMessage(content='what is 2+2'),
 AIMessage(content='2+2 = 4'),
 HumanMessage(content='Thanks'),
 AIMessage(content='No problem, what more can I help you with!'),
 HumanMessage(content='having fun'),
 AIMessage(content='Yes!')]

In [48]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt|llm_model)
response = chain.invoke({"messages": messages + [HumanMessage(content="What is 2+2")]})
response.content

'2 + 2 = 4 😊  \n\nIs there anything else I can help you with?  \n'

In [55]:
with_msg_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable": {"session_id": "chat5"}}

response = with_msg_history.invoke(
    {"messages": messages + [HumanMessage(content="What is 2+2")]},
    config=config
)

response.content

'2 + 2 = 4  😊 \n\nIs there anything else I can help you with?\n'